In [20]:
from bs4 import BeautifulSoup
import requests
import pprint
import re
import datetime
import pandas as pd
import os
import shutil
NO_OF_WEEKS_TO_SCRAPE=1

In [21]:
room_df_dict = {}

In [22]:
cookie={
    "PHPSESSID":"sv8vsjuk8ocbvkbsdmunvmsil4"
}
url="http://roombooking.iitd.ac.in/book/" #add br(i) to this

req_header = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Origin': 'http://roombooking.iitd.ac.in',
    'Upgrade-Insecure-Requests': '1',
    'Content-Type': 'application/x-www-form-urlencoded',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Referer': 'http://roombooking.iitd.ac.in/book/book_room',
    'Accept-Encoding': 'gzip, deflate',
    'Accept-Language': 'en-US,en;q=0.9',
}

capacity = '1' #minimum capacity allows show of all rooms
rci = '0' #room category id
uniq_room_list = [] # The list of all rooms
# The table shown starts from max((input_date - 3),(today+1))
#So intervals must be tomorrow;tomorrow+7+3;...
date_list = []
n = datetime.datetime.now()
for i in range(NO_OF_WEEKS_TO_SCRAPE):
    n = datetime.datetime.now()+datetime.timedelta(days=(i*7+4))
    date_list.append(str(n.year)+"-"+str(n.month)+"-"+str(n.day))  

br1 = "book_room"
br2 = "show_rooms?capacity="+capacity+"&room_category_id="+rci


In [5]:
w = requests.get(url+br2,cookies=cookie,headers=req_header,data={'capacity':capacity,'room_category_id':rci})
room_list = w.text
match = r"LH [0-9]{3}"
pattern = re.compile(match)
dup_room_list = pattern.findall(room_list)
for i in dup_room_list:
    if i not in uniq_room_list:
        uniq_room_list.append(i)

In [6]:
data = {'capacity':capacity, 'room_category_id':rci, 'room_no':'','date':''}

In [7]:
def data_cleaner(needed_table):
    l=needed_table.find_all('tr')
    for k in l:
        m = k.find_all('td')
        if (m == []):
            m = k.find_all('th')
        else:
            for z in m:
                if (z.a['class'][0] == "alloted"):
                    cont = z.a['title']
                    z.a.append(cont[:6])
                    del z.a["title"]
                elif (z.a['class'][0] == "available"):
                    z.a.append("Free")
                    del z.a["id"]
                else:
                    z.a.append("Not Allowed")
                    del z.a["title"]
                    del z.a["id"]
    return needed_table

In [8]:
with requests.Session() as s:
    for i in date_list: #Take a date
        for j in uniq_room_list: #Take a room
#             j = uniq_room_list[0]
            data['room_no']=j
            data['date']=i
#             r = requests.get(url=(url+br1),cookies=cookie)
            r = requests.post(url=(url+br1),cookies=cookie,headers=req_header,data=data)
            soup = BeautifulSoup(r.content)
            
            all_tables = soup.find_all('table')
            actual_table = all_tables[1]
            cleaned = data_cleaner(actual_table)
            to_str_cleaned = str(cleaned)
            df = pd.read_html(to_str_cleaned)[0]
#             print(type(df))
            dic = df.to_dict('list')
            room_df_dict[j]=dic
#             req_table = str(soup.find_all('table')[1])
#             with open(j+".html","w") as f:
#                 f.write(req_table)

    

In [9]:
room_df_dict

{'LH 518': {'Time': ['08:00',
   '08:30',
   '09:00',
   '09:30',
   '10:00',
   '10:30',
   '11:00',
   '11:30',
   '12:00',
   '12:30',
   '13:00',
   '13:30',
   '14:00',
   '14:30',
   '15:00',
   '15:30',
   '16:00',
   '16:30',
   '17:00',
   '17:30',
   '18:00',
   '18:30',
   '19:00',
   '19:30'],
  '2019-03-19(Tuesday)': ['PYL304',
   'PYL304',
   'CLL402',
   'CLL402',
   'BML740',
   'BML740',
   'BML750',
   'BML750',
   'BBL443',
   'BBL443',
   'HUL286',
   'HUL286',
   'COL886',
   'COL886',
   'COL886',
   'ASL738',
   'ASL738',
   'ASL738',
   'ELL402',
   'ELL402',
   'Free',
   'Free',
   'Free',
   'Free'],
  '2019-03-20(Wednesday)': ['PYL304',
   'PYL304',
   'CLL402',
   'CLL402',
   'BML740',
   'BML740',
   'BBL341',
   'BBL341',
   'ELL402',
   'ELL402',
   'Free',
   'Free',
   'Free',
   'Free',
   'ESQ301',
   'ESQ301',
   'ESQ301',
   'ESQ301',
   'ESQ303',
   'ESQ303',
   'ESQ303',
   'ESQ303',
   'Free',
   'Free'],
  '2019-03-21(Thursday)': ['Free',
   '

In [10]:
def getTimeList(room_df_dict):
    return room_df_dict[list(room_df_dict.keys())[0]]['Time']

In [11]:
def getRoomList(room_df_dict):
    return list(room_df_dict.keys())

In [12]:
def getDayList(room_df_dict):
    return list((room_df_dict[list(room_df_dict.keys())[0]]).keys())[1:]

In [13]:
#room_df_dict is of the format Room:{Day1/Time:[Course1,Course2,...],Day2/Time:[...]}
#The function must return Day:{Room:[Course1,Course2,...]}
def freeRoomsPerDay(room_df_dict):
    dic_to_return= {}
    rooms_list=getRoomList(room_df_dict)
    days_list=getDayList(room_df_dict)
    for currDay in days_list:
        dic_ele={}
        for currRoom in rooms_list:
            courses_list=room_df_dict[currRoom][currDay]
            dic_ele[currRoom]=courses_list
        dic_to_return[currDay]=dic_ele
    return dic_to_return    

In [14]:
def dirStructureSet(room_df_dict):
    
    try:
        os.mkdir("dayWiseTables")
    except FileExistsError:
        pass
    finally:
        try:
            os.mkdir("smallTables")
        except FileExistsError:
            shutil.rmtree("smallTables")
            os.mkdir("smallTables")
        finally:
            for i in getDayList(room_df_dict):
                pattern = re.compile(r'[0-9]{4}-[0-9]{1,2}-([0-9]{1,2})\((.*)\)')
                m = pattern.findall(i)
                try:
                    os.mkdir("smallTables/"+m[0][0]+m[0][1])
                except FileExistsError:
                    continue

    

In [15]:
def createDFfromDictLarge(new_room_df_dict,old_room_df_dict,folder):
    for i in list(new_room_df_dict.keys()):
        currDay=new_room_df_dict[i]
        df_from_dict=pd.DataFrame.from_dict(currDay,orient='index',columns=getTimeList(old_room_df_dict))
        pattern = re.compile(r'[0-9]{4}-[0-9]{1,2}-([0-9]{1,2})\((.*)\)')
        m = pattern.findall(i)
        df_from_dict.to_html(folder+"/"+m[0][0]+m[0][1]+".html")

In [16]:
def createDFfromDictSmall(new_room_df_dict,folder):
    for i in list(new_room_df_dict.keys()):
        currDay=new_room_df_dict[i]
        for j in list(currDay.keys()):
            currTime=currDay[j]
            pattern = re.compile(r'[0-9]{4}-[0-9]{1,2}-([0-9]{1,2})\((.*)\)')
            m = pattern.findall(i)
            df_from_dict=pd.DataFrame.from_dict({i:currTime},orient='columns')
            df_from_dict.to_html(folder+"/"+m[0][0]+m[0][1]+"/"+j+".html")

In [17]:
# Of the format Day:{Time:[Free_room1,Free_room2,...]}
def roomsPerHour(room_df_dict):
    dic_to_return={}
    rooms_list=getRoomList(room_df_dict)
    days_list=getDayList(room_df_dict)
    time_list=getTimeList(room_df_dict)
    for j in days_list:
        ele ={}
        for i in time_list:
            ele[i]=[]
        dic_to_return[j]=ele
#     print (room_df_dict)
    for currDay in days_list:
        for currRoom in rooms_list:
            for i in range(len(time_list)):
                currTime=time_list[i]
                if (room_df_dict[currRoom][currDay][i] == 'Free'):
                    dic_to_return[currDay][currTime].append(currRoom)
    return dic_to_return

In [18]:
tl=getTimeList(room_df_dict)
d1=freeRoomsPerDay(room_df_dict)
d2=roomsPerHour(room_df_dict)
dirStructureSet(room_df_dict)
createDFfromDictLarge(d1,room_df_dict,"dayWiseTables")
createDFfromDictSmall(d2,"smallTables")

(2019, 3, 16)

['2019-3-20', '2019-3-27']